# Spatial Optimization

In [41]:
import folium
import pandas as pd
import numpy as np
import geopandas as gpd
from pyproj import CRS
from shapely.geometry import Point
from sklearn.neighbors import KernelDensity
from statsmodels.nonparametric.bandwidths import bw_silverman

## Configure Parameters

In [42]:
bus_stops_weight = 0.5
hospitals_weight = 1.0
restaurants_weight = 0.25
num_bpoints = 5  # number of best points required

## Load Data

In [43]:
bus_stops_file = './bus_stops.csv'
hospitals_file = './hospitals.csv'
restaurants_file = './restaurants.csv'
bus_stops_data = pd.read_csv(bus_stops_file)
hospitals_data = pd.read_csv(hospitals_file)
restaurants_data = pd.read_csv(restaurants_file)

## Prepare Data

In [44]:
bus_stops_data['weights'] = bus_stops_weight
bus_stops_data['location_type'] = 'bus_stop'
hospitals_data['weights'] = hospitals_weight
hospitals_data['location_type'] = 'hospital'
restaurants_data['weights'] = restaurants_weight
restaurants_data['location_type'] = 'restaurant'
datasets = [bus_stops_data, hospitals_data, restaurants_data]
merged_data = pd.concat(datasets)

# convert latitude and longitude from degrees to radians
merged_data.loc[:, 'latitude'] = merged_data.loc[:, 'latitude']*np.pi/180
merged_data.loc[:, 'longitude'] = merged_data.loc[:, 'longitude']*np.pi/180

weights = merged_data['weights']
display(merged_data.head())

,id,latitude,longitude,weights,location_type
0,d461dd24beb4b0ef4eb8f5251f14a133,0.226120,1.356476,0.5,bus_stop
1,567e9aa5e9849f42f6161bf6b73bd47c,0.224215,1.355625,0.5,bus_stop
2,cf2fd7e978ea8f70b790c931cb6c8ccf,0.225834,1.356039,0.5,bus_stop
3,ca41840db8acbbc8a10e6e4c69d91c36,0.226040,1.352490,0.5,bus_stop
4,219773e23e2d7659a5935bb63e75347f,0.230344,1.357465,0.5,bus_stop


## Kernel Density Estimation

In [45]:
silverman_bandwidth = max(bw_silverman(
    merged_data[['latitude', 'longitude']].to_numpy()))
kde = KernelDensity(bandwidth=silverman_bandwidth,
                    algorithm='ball_tree', metric='haversine')
trained_estimator = kde.fit(
    merged_data[['latitude', 'longitude']].to_numpy(), weights.to_numpy())

## Finding Best Coordinates

In [48]:
bpoint_indices = np.argsort(trained_estimator.score_samples(
    merged_data[['latitude', 'longitude']].to_numpy()))[-num_bpoints:]
bpoint_indices = np.flip(bpoint_indices)  # order descendingly
best_locations = merged_data.iloc[bpoint_indices]

# convert latitude and longitude from radians to degrees
best_locations.loc[:, 'latitude'] = best_locations.loc[:, 'latitude']*180/np.pi
best_locations.loc[:, 'longitude'] = best_locations.loc[:,
                                                        'longitude']*180/np.pi

best_locations.index = range(1, 6)  # show the ranks of the locations
display(best_locations)

/home/braceletboy/.local/share/virtualenvs/Random-zvK-jQge/lib/python3.6/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,id,latitude,longitude,weights,location_type
1,c5a9e9be83c054fad5d40b489b14cf3b,12.948173,77.570592,0.50,bus_stop
2,06bf01c8147dc25380f0f74d571f6166cd4853d7,12.944839,77.571819,0.25,restaurant
3,3f2d7880c41532bb1d38b9e71462b5255d367313,12.948411,77.572026,1.00,hospital
4,677de0c10f058704c1f2b5a83fbe49c0fdd40d06,12.945072,77.571343,0.25,restaurant
5,95bb888c22be664fe4c47ce9f2acbe75367218d9,12.948033,77.568855,0.25,restaurant


## Visualization

In [49]:
# convert locations to points ('x' is longitude and 'y' is latitude)
best_points = [Point(location) for location in zip(
    best_locations['longitude'], best_locations['latitude'])]
gdf = gpd.GeoDataFrame({
    'id': best_locations['id'].to_numpy(),
    'location_type': best_locations['location_type'].to_numpy(),
    'geometry': best_points
})  # geo dataframe

# set crs - coordinate reference system
gdf.crs = CRS.from_epsg(4326)  # latitude longitude system
# convert to mercator system because our map is a mercator map
gdf.to_crs(CRS.from_epsg(3395), inplace=True)

# map it
map_plot = folium.Map(location=[
    np.mean(best_locations['latitude'].to_numpy()),
    np.mean(best_locations['longitude'].to_numpy())], zoom_start=14)
points_gjson = folium.features.GeoJson(gdf['geometry'], name='best_locations')
points_gjson.add_to(map_plot)
map_plot